In [ ]:
# BASE CASE: load the viewer with desired options
from vedo import *

# DEBUG MODE, not using pip install here
import os
import sys
sys.path.append(os.path.abspath('../'))
from iblviewer import *

resolution = 25 # units = um
controller = atlas_controller.AtlasController()
controller.initialize(resolution, embed_ui=True, jupyter=True)
controller.render()

In [ ]:
def add_point_neurons(controller, data, with_labels=False):
    """
    Add point neurons
    :param controller: IBLViewer atlas controller
    :param data: At least 5D array of x, y, z, region, custom data. 
    All the columns after 'region' are be stored as a time series
    :param with_labels: Whether labels are added to the points
    """
    if isinstance(data, str):
        pickles = []
        data_path = data
        with (open(os.path.abspath(data_path), 'rb')) as openfile:
            while True:
                try:
                    pickles.append(pickle.load(openfile))
                except EOFError:
                    break
        data = pickles[0]

    # Structure of 'data': x | y | z | region | pre-time allegiance | during-time allegiance | post-time allegiance 
    positions = []
    regions = []
    timings = []
    # Cleaning weird data and extracting what we need. When Alessandro fixes his code, we can get rid of this cleaning.
    for weird_data in data:
        try:
            positions.append([weird_data[0][0], weird_data[1][0], weird_data[2][0]])
        except TypeError:
            continue
            #positions.append(bad_stuff[:3])
        regions.append(weird_data[3])
        timings.append(weird_data[4:])
    positions = np.array(positions).astype(np.float)

    timings = np.array(timings)
    regions = np.array(regions)
    # 16um is a good compromise for visibility from afar. So we make somata roughly 2-3 times larger than they are in reality
    points = controller.view.new_points(positions, radius=16, values=timings, as_spheres=True, noise_amount=100)
    
    actors = [points]
    if with_labels:
        # Using functions from vedo (easy-to-use wrapper on top of VTK)
        labels = points.labels('id', cells=True)
        actors.append(labels)
    controller.plot.add(actors)
    return points